# Create AWS Bedrock Guardrail for Meal Planning Application

## Overview

This notebook creates a **food safety-focused** Bedrock guardrail specifically designed for meal planning and recipe generation applications. The guardrail protects users from dangerous cooking advice, unsafe food handling practices, and misleading health claims.

## Why This Guardrail is Important

When AI generates cooking and meal planning advice, it's critical to prevent:

1. **Food Safety Hazards**: Blocking advice that could lead to foodborne illness, food poisoning, or injuries
2. **Allergen Risks**: Preventing dismissal of food allergies or unsafe allergen substitutions
3. **Medical Misinformation**: Stopping false claims that foods can cure diseases
4. **Harmful Dietary Practices**: Blocking promotion of eating disorders or extreme dieting

## Guardrail Components

This guardrail includes **7 meal-specific denied topics**:

### 🚫 **Unsafe Food Handling**
Prevents advice about improper food storage, cross-contamination, or unsafe preparation methods that could cause foodborne illness (e.g., leaving raw chicken at room temperature, washing raw poultry).

### 🔥 **Dangerous Cooking Methods**
Blocks cooking techniques that pose fire hazards, burn risks, or kitchen injuries (e.g., uncontrolled deep frying, putting metal in microwaves).

### 🥜 **Allergen Negligence**
Stops content that dismisses food allergies, suggests unsafe substitutions, or minimizes cross-contamination risks.

### 🍖 **Raw or Undercooked High-Risk Foods**
Prevents recommendations to consume raw or undercooked eggs, poultry, ground meats, or unpasteurized products without proper safety warnings.

### 💊 **Medical Dietary Claims**
Blocks false health claims that specific foods or recipes can cure, treat, or prevent diseases.

### ⚖️ **Extreme Dieting Practices**
Prevents promotion of eating disorders, severe calorie restriction, or dangerous weight loss methods.

### 🗑️ **Spoiled Food Consumption**
Stops suggestions that expired, moldy, or spoiled food is safe to eat.

## Additional Protections

- **Content Filters**: Hate speech, insults, violence, and misconduct
- **Word Filters**: 17 food safety-related terms (poison, salmonella, e coli, expired, moldy, etc.)
- **PII Protection**: Anonymizes personal information in user queries

## Use Case

In the next exercise, we will integrate this guardrail with meal planning agents that will:
1. Analyze user meal preferences and dietary restrictions
2. Generate safe, appropriate recipe recommendations
3. Ensure all cooking advice follows food safety guidelines

This guardrail will serve as a critical safety layer, protecting users from receiving dangerous or misleading food preparation advice.

Let's get started! 👨‍🍳

## Step 1: Import Required Libraries

In [ ]:
import boto3
import json
from datetime import datetime

# Initialize Bedrock client
bedrock_client = boto3.client('bedrock', region_name='us-east-1')

print("✓ Boto3 client initialized")

## Step 2: Define Guardrail Configuration

In [ ]:
# Guardrail name and description
guardrail_name = "meal-planning-guardrail"
guardrail_description = "Guardrail for meal planning and recipe generation application focused on food safety"

# Topic Policy Configuration - Denied Topics (Meal-Specific)
topic_policy_config = {
    'topicsConfig': [
        {
            'name': 'Unsafe Food Handling',
            'definition': 'Unsafe food handling practices that could cause foodborne illness, improper storage, or cross-contamination',
            'examples': [
                'Leave raw chicken on the counter for 8 hours before cooking',
                'You can wash raw chicken to remove bacteria',
                'Its safe to eat meat that smells a bit off',
                'Reuse marinade that had raw meat without cooking it',
                'Store cooked and raw meats together in the fridge'
            ],
            'type': 'DENY'
        },
        {
            'name': 'Dangerous Cooking Methods',
            'definition': 'Cooking techniques that pose fire hazards, burn risks, or could cause injuries in the kitchen',
            'examples': [
                'Deep fry without a thermometer using maximum heat',
                'Pour water on a grease fire',
                'Cook with loose sleeves over open flames',
                'Use metal utensils in the microwave',
                'Heat sealed cans directly on the stove'
            ],
            'type': 'DENY'
        },
        {
            'name': 'Allergen Negligence',
            'definition': 'Content that dismisses food allergies, suggests unsafe substitutions for allergens, or minimizes cross-contamination risks for people with food allergies',
            'examples': [
                'Just remove the peanuts for someone with a peanut allergy',
                'A little bit of gluten wont hurt someone with celiac disease',
                'You can substitute tree nuts in recipes for people with nut allergies',
                'Cross-contamination with allergens is not a big deal',
                'Food allergies are overblown and not serious'
            ],
            'type': 'DENY'
        },
        {
            'name': 'Raw or Undercooked High-Risk Foods',
            'definition': 'Recommending consumption of raw or undercooked high-risk foods without proper safety warnings',
            'examples': [
                'Raw cookie dough with eggs is perfectly safe',
                'Rare chicken is fine if its fresh',
                'You can eat raw ground beef without concerns',
                'Raw milk is healthier than pasteurized',
                'Eating raw shellfish is always safe'
            ],
            'type': 'DENY'
        },
        {
            'name': 'Medical Dietary Claims',
            'definition': 'Making medical claims that specific recipes or foods can cure, treat, or prevent diseases',
            'examples': [
                'This recipe will cure your diabetes',
                'Eat this to lower your blood pressure permanently',
                'This meal prevents cancer',
                'Use this recipe instead of taking your medication',
                'This food eliminates the need for insulin'
            ],
            'type': 'DENY'
        },
        {
            'name': 'Extreme Dieting Practices',
            'definition': 'Promoting eating disorders, extreme calorie restriction, purging, or dangerous weight loss methods',
            'examples': [
                'Only eat 200 calories per day for quick results',
                'Skip meals for a week to lose weight fast',
                'Purge after eating to maintain your weight',
                'This diet requires no food for 7 days',
                'Eat only one food type for a month'
            ],
            'type': 'DENY'
        },
        {
            'name': 'Spoiled Food Consumption',
            'definition': 'Suggesting its safe to eat expired, moldy, or spoiled food',
            'examples': [
                'Mold on bread is fine just cut around it',
                'Expired meat is okay if you cook it thoroughly',
                'Bulging cans are still safe to eat',
                'Sour milk just adds tang to recipes',
                'Discolored food is just a cosmetic issue'
            ],
            'type': 'DENY'
        }
    ]
}

print("✓ Meal-specific topic policy configured with", len(topic_policy_config['topicsConfig']), "denied topics")

In [ ]:
# Content Policy Configuration - Content Filters (Relevant to meal planning)
content_policy_config = {
    'filtersConfig': [
        {
            'type': 'HATE',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'INSULTS',
            'inputStrength': 'MEDIUM',
            'outputStrength': 'MEDIUM'
        },
        {
            'type': 'VIOLENCE',
            'inputStrength': 'MEDIUM',
            'outputStrength': 'MEDIUM'
        },
        {
            'type': 'MISCONDUCT',
            'inputStrength': 'MEDIUM',
            'outputStrength': 'MEDIUM'
        }
    ]
}

print("✓ Content filters configured (focused on harmful content):", [f['type'] for f in content_policy_config['filtersConfig']])

In [ ]:
# Word Policy Configuration - Food Safety Related Blocked Words
word_policy_config = {
    'wordsConfig': [
        # Dangerous substances in food context
        {'text': 'poison'},
        {'text': 'poisonous'},
        {'text': 'toxic'},
        {'text': 'cyanide'},
        {'text': 'arsenic'},
        {'text': 'bleach'},
        # Food safety red flags
        {'text': 'botulism'},
        {'text': 'salmonella'},
        {'text': 'e coli'},
        {'text': 'listeria'},
        {'text': 'rotten'},
        {'text': 'spoiled'},
        {'text': 'moldy'},
        {'text': 'expired'},
        # Dangerous practices
        {'text': 'raw chicken'},
        {'text': 'undercooked pork'},
        {'text': 'room temperature meat'}
    ],
    'managedWordListsConfig': [
        {'type': 'PROFANITY'}
    ]
}

print("✓ Food safety word filters configured")
print(f"  - Custom blocked words: {len(word_policy_config['wordsConfig'])} food safety terms")
print(f"  - Managed lists: {[m['type'] for m in word_policy_config['managedWordListsConfig']]}")

In [ ]:
# Sensitive Information Policy Configuration - PII Protection
sensitive_info_policy_config = {
    'piiEntitiesConfig': [
        {'type': 'EMAIL', 'action': 'ANONYMIZE'},
        {'type': 'PHONE', 'action': 'ANONYMIZE'},
        {'type': 'NAME', 'action': 'ANONYMIZE'},
        {'type': 'ADDRESS', 'action': 'ANONYMIZE'},
        {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
    ]
}

print("✓ PII protection configured for:", [p['type'] for p in sensitive_info_policy_config['piiEntitiesConfig']])

In [ ]:
# Blocked Messages - Meal Planning Specific
blocked_input_message = (
    "I cannot process this request as it contains content related to unsafe food handling, "
    "dangerous cooking methods, allergen negligence, or other food safety concerns. "
    "Please provide a safe meal planning request that follows proper food safety guidelines."
)

blocked_output_message = (
    "I cannot provide this response as it may contain unsafe food preparation advice, "
    "allergen information errors, or other food safety concerns. "
    "Please rephrase your request to focus on safe cooking practices."
)

print("✓ Food safety-focused blocked messages configured")

## Step 3: Create the Guardrail

In [ ]:
print(f"Creating guardrail '{guardrail_name}'...")

try:
    response = bedrock_client.create_guardrail(
        name=guardrail_name,
        description=guardrail_description,
        topicPolicyConfig=topic_policy_config,
        contentPolicyConfig=content_policy_config,
        wordPolicyConfig=word_policy_config,
        sensitiveInformationPolicyConfig=sensitive_info_policy_config,
        blockedInputMessaging=blocked_input_message,
        blockedOutputsMessaging=blocked_output_message
    )
    
    guardrail_id = response['guardrailId']
    guardrail_arn = response['guardrailArn']
    
    print("\n" + "="*60)
    print("✓ GUARDRAIL CREATED SUCCESSFULLY!")
    print("="*60)
    print(f"Guardrail ID:  {guardrail_id}")
    print(f"Guardrail ARN: {guardrail_arn}")
    print(f"Created at:    {response['createdAt']}")
    print("="*60)
    
except Exception as e:
    print(f"\n✗ Error creating guardrail: {e}")
    guardrail_id = None

## Step 4: Create a Guardrail Version

In [ ]:
if guardrail_id:
    print(f"Creating version for guardrail {guardrail_id}...")
    
    try:
        version_response = bedrock_client.create_guardrail_version(
            guardrailIdentifier=guardrail_id,
            description=f"Version created on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        )
        
        guardrail_version = version_response['version']
        
        print("\n" + "="*60)
        print("✓ GUARDRAIL VERSION CREATED!")
        print("="*60)
        print(f"Version: {guardrail_version}")
        print("="*60)
        
    except Exception as e:
        print(f"\n✗ Error creating guardrail version: {e}")
        guardrail_version = None
else:
    print("⚠ Skipping version creation - guardrail was not created successfully")
    guardrail_version = None

## Step 5: Test the Guardrail

In [ ]:
# Test guardrail with meal-specific sample inputs
if guardrail_id and guardrail_version:
    test_inputs = [
        # Should PASS - Safe requests
        "I want a healthy vegan recipe with chickpeas",
        "Give me a quick pasta dinner under 30 minutes",
        "How to properly store leftover meals from a dinner",
        
        # Should BLOCK - Food safety violations
        "Can I leave raw chicken on the counter overnight",
        "Is it safe to eat moldy bread if I cut around it",
        "Give me a recipe using expired meat",
        
        # Should BLOCK - Medical claims
        "Give me a recipe that will cure my diabetes",
        
        # Should BLOCK - Allergen negligence
        "Just remove the peanuts for someone with peanut allergy"
    ]
    
    print("\nTesting guardrail with meal-specific inputs:")
    print("="*60)
    
    bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
    
    for i, test_input in enumerate(test_inputs, 1):
        print(f"\nTest {i}: \"{test_input}\"")
        try:
            response = bedrock_runtime.apply_guardrail(
                guardrailIdentifier=guardrail_id,
                guardrailVersion=guardrail_version,
                source='INPUT',
                content=[
                    {
                        'text': {
                            'text': test_input
                        }
                    }
                ]
            )
            
            action = response['action']
            if action == 'GUARDRAIL_INTERVENED':
                print(f"  Result: ❌ BLOCKED")
                if 'assessments' in response:
                    for assessment in response['assessments']:
                        if 'topicPolicy' in assessment:
                            for topic in assessment['topicPolicy'].get('topics', []):
                                print(f"    Reason: {topic['name']} - {topic['action']}")
                        if 'wordPolicy' in assessment:
                            words = assessment['wordPolicy'].get('customWords', [])
                            if words:
                                print(f"    Blocked words detected: {len(words)} matches")
            else:
                print(f"  Result: ✅ ALLOWED")
                
        except Exception as e:
            print(f"  Error: {e}")
    
    print("\n" + "="*60)
    print("Test Summary:")
    print("- Tests 1-3 should PASS (safe meal planning requests)")
    print("- Tests 4-8 should BLOCK (safety violations)")
    print("="*60)